In [ ]:
# Import libraries
import os
import magpy as mp
import scanpy as sc
import numpy as np
import pandas as pd
import anndata as ad
import diffxpy.api as de
import matplotlib.pyplot as plt

expt_path = "/proj/magness/"
cds010_path = "/proj/magness/CDS010_hashtag"
cds014_path = "/proj/magness/CDS014_hashtag"
cds015_path = "/proj/magness/CDS015_hashtag"
combined_path = "/proj/magness/CDS010-014-015_combined"

figdir = combined_path+'/figures/computational_supp_figs/'
sc.settings.figdir = figdir

In [ ]:
### COPY AND ANNOTATE RAW DATA ###

# mp.pipeline.copy_data(expt_path=expt_path, sample_ID='m-014')
# mp.pipeline.copy_data(expt_path=expt_path, sample_ID='m-019')
# mp.pipeline.copy_data(expt_path=expt_path, sample_ID='m-021')

# mp.pipeline.annotate(cds010_path, sample_ID='m-014',hash_clusters=7, save = False, write_pfx = 'Donor2_CDS010')
# mp.pipeline.annotate(cds014_path, sample_ID='m-019',hash_clusters=6, save = False)
mp.pipeline.annotate(cds015_path, sample_ID='m-021',hash_clusters=7, save = False)

In [ ]:
### PREPROCESS AND FILTER/PREPROCESS DATA ###

## FILTER ##

# #CDS010: Filter out negative cells
adata10 = mp.load(cds010_path,1)
print(adata10)
adata10 = adata10[adata10.obs['hash_label'] != 'Negative',:]
print(adata10)

# #CDS014: Call Negative cells in hash_cluster 0 as transverse colon, filter out remaining negatives
adata14 = mp.load(cds014_path,1)
# print(adata14)
tc_mask = (adata14.obs['hash_label'] == 'Negative') & (adata14.obs['hash_cluster'] == 0)
adata14.obs['hash_label'] = adata14.obs['hash_label'].mask(tc_mask,'TotalSeq-B0256')
adata14 = adata14[adata14.obs['hash_label'] != 'Negative',:]
# print(adata14)

# # #CDS015: Filter out negative cells
adata15 = mp.load(cds015_path,1)
# print(adata15)
adata15 = adata15[adata15.obs['hash_label'] != 'Negative',:]
# print(adata15)


## PREPROCESS AND SAVE ##

#CDS010
# mp.settings.max_percent_mito = 50
# mp.settings.min_counts_per_cell = 1000
# mp.settings.max_counts_per_cell = 30000
# mp.settings.min_genes_per_cell = 800
# mp.pipeline.preprocess(cds010_path, data=adata10, sample_ID='m-014', save = False)

# #CDS014
# mp.settings.min_counts_per_cell = 3000
# mp.settings.max_counts_per_cell = 50000
# mp.settings.max_percent_mito = 75
# mp.settings.min_genes_per_cell = 500
mp.pipeline.preprocess(cds014_path, data=adata14, sample_ID='m-019', save = False)

# #CDS015
# mp.settings.min_counts_per_cell = 3000
# mp.settings.max_counts_per_cell = 50000
# mp.settings.max_percent_mito = 75
# mp.settings.min_genes_per_cell = 500
mp.pipeline.preprocess(cds015_path, data=adata15, sample_ID='m-021', save = False)

In [ ]:
### DISPLAY COUNTS AFTER FILTERING

def count(series):
    unique, counts = np.unique(series.to_numpy(), return_counts=True)
    return(dict(zip(unique, counts)))

adata10 = mp.load(cds010_path,2)
adata14 = mp.load(cds014_path,2)
adata15 = mp.load(cds015_path,2)

print('Donor 10 counts per region')
foo = count(adata10.obs['hash_label'])
totalfoo = 0
for key in foo:
    print(f"{key} - {foo[key]}")
    totalfoo += foo[key]
print(f"Total - {totalfoo}")
print()

print('Donor 14 counts per region')
bar = count(adata14.obs['hash_label'])
totalbar = 0
for key in bar:
    print(f"{key} - {bar[key]}")
    totalbar += bar[key]
print(f"Total - {totalbar}")
print()

print('Donor 15 counts per region')
bar = count(adata15.obs['hash_label'])
totalbar = 0
for key in bar:
    print(f"{key} - {bar[key]}")
    totalbar += bar[key]
print(f"Total - {totalbar}")

In [ ]:
### COMBINE DONORS AND SAVE ###

adata10 = mp.load(cds010_path,2)
adata14 = mp.load(cds014_path,2)
adata15 = mp.load(cds015_path,2)

adata10.obs['donor'] = "CDS010"
donor10_map = {
    'TotalSeq-B0251':"Duo",
    'TotalSeq-B0252':"Jej",
    'TotalSeq-B0253':"Ile",
    'TotalSeq-B0254':"AC",
    'TotalSeq-B0255':"TC",
    'TotalSeq-B0256':"DC"}
adata10.obs['region'] = adata10.obs['hash_label'].map(donor10_map)

adata14.obs['donor'] = "CDS014"
donor14_map = {
    'TotalSeq-B0251':"Duo",
    'TotalSeq-B0252':"Jej",
    'TotalSeq-B0253':"Ile",
    'TotalSeq-B0254':"AC",
    'TotalSeq-B0255':"DC",
    'TotalSeq-B0256':"TC"}
adata14.obs['region'] = adata14.obs['hash_label'].map(donor14_map)

adata15.obs['donor'] = "CDS015"
donor15_map = {
    'TotalSeq-B0251':"Duo",
    'TotalSeq-B0252':"Jej",
    'TotalSeq-B0253':"Ile",
    'TotalSeq-B0254':"AC",
    'TotalSeq-B0255':"DC",
    'TotalSeq-B0256':"TC"}
adata15.obs['region'] = adata15.obs['hash_label'].map(donor15_map)

adata = adata10.concatenate(adata14, adata15, join='outer')
mp.save(adata,combined_path,2)

In [ ]:
### PROCESS DATASETS ###
##tODO: Reprocess datasets using standardized normalization factor

# mp.pipeline.process(cds010_path)
# mp.pipeline.process(cds010_path,regress_cell_cycle=True,write_file="processed_adata_cc-regressed.h5ad")

# mp.pipeline.process(cds014_path)
# mp.pipeline.process(cds014_path,regress_cell_cycle=True,write_file="processed_adata_cc-regressed.h5ad")

# mp.pipeline.process(combined_path, write_file="combined_processed_adata.h5ad")
# mp.pipeline.process(combined_path, regress_cell_cycle=True, write_file="combined_processed_adata_cc-regressed.h5ad")

In [ ]:
### PROCESS AND CLUSTER SEPARATE DATA ###

rf = "combined_processed_adata_cc-regressed.h5ad"
mp.settings.leiden_resolution = 0.8
mp.sett


# mp.pipeline.cluster(cds010_path)
# mp.pipeline.cluster(cds014_path)
# mp.pipeline.cluster(cds015_path)

mp.settings.num_neighbors = 10
mp.pipeline.cluster(combined_path,read_file=rf,write_file="clustered_adata_k10.h5ad",harmonize=True)

mp.settings.num_neighbors = 25
mp.pipeline.cluster(combined_path,read_file=rf,write_file="clustered_adata_k25.h5ad",harmonize=True)

mp.settings.num_neighbors = 50
mp.pipeline.cluster(combined_path,read_file=rf,write_file="clustered_adata_k50.h5ad",harmonize=True)


In [ ]:
### DISPLAY PER-DONOR CONTRIBUTIONS ###

adata = mp.load(combined_path,"clustered_adata_k25.h5ad")

sc.pl.umap(adata,color='leiden')
sc.pl.umap(adata,color=['leiden','phase','region','donor'],ncols=2)

for donor in adata.obs['donor'].unique():
    print(donor)
    subset = adata[adata.obs['donor']==donor,:]
    sc.pl.umap(subset,color=['region'])